<a href="https://colab.research.google.com/github/PratypartyY2K/InstructBLIP_Improvement/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
!pip install open3d

  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 65.8 MB/s  0:00:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 188.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 112.3 MB/s  0:00:00
Using cached jedi-0.19.2-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [open3d]


In [8]:
import os
import glob

# 1. Find the Config File
# The file is usually named 'blip2_instruct_vicuna7b.yaml'
config_pattern = "/content/LAVIS/lavis/configs/models/blip2/blip2_instruct_vicuna7b.yaml"
config_files = glob.glob(config_pattern)

if not config_files:
    # Fallback search if exact path differs
    config_files = glob.glob("/content/LAVIS/**/blip2_instruct_vicuna7b.yaml", recursive=True)

if not config_files:
    raise FileNotFoundError("Could not find the model config file! Please check /content/LAVIS.")

config_path = config_files[0]
print(f"Found config file at: {config_path}")

# 2. Read and Patch the File
with open(config_path, "r") as f:
    content = f.read()

# The config likely contains: llm_model: "./llm/vicuna-7b"
# We need to change it to:   llm_model: "lmsys/vicuna-7b-v1.1"

if '"./llm/vicuna-7b"' in content or "'./llm/vicuna-7b'" in content:
    print("Patching local path to Hugging Face URL...")
    new_content = content.replace('"./llm/vicuna-7b"', '"lmsys/vicuna-7b-v1.1"')
    new_content = new_content.replace("'./llm/vicuna-7b'", '"lmsys/vicuna-7b-v1.1"')

    # Save the change
    with open(config_path, "w") as f:
        f.write(new_content)
    print("Config patched successfully!")
else:
    print("Config already seems correct or format is different.")
    # Force print a snippet to debug if patch failed
    print("Preview of config:")
    print(content[:500])

# 3. RETRY LOAD (Immediately after patching)
print("\nRetrying Model Load...")
import torch
from lavis.models import load_model_and_preprocess

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_vicuna_instruct",
    model_type="vicuna7b",
    is_eval=True,
    device=device
)
print("\nSUCCESS: InstructBLIP Loaded! NOW run your Main Evaluation Script.")

Found config file at: /content/LAVIS/lavis/configs/models/blip2/blip2_instruct_vicuna7b.yaml
Patching local path to Hugging Face URL...
Config patched successfully!

Retrying Model Load...


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2.11G/2.11G [01:55<00:00, 19.7MB/s]



SUCCESS: InstructBLIP Loaded! NOW run your Main Evaluation Script.


In [10]:
import zipfile
import os
from google.colab import drive

# 1. Mount Google Drive
# This will prompt for authorization to access your Drive
drive.mount('/content/drive')

# 2. Define the paths
zip_path = '/content/drive/MyDrive/flickr8k.zip'
extract_to_path = '/content/drive/MyDrive/flickr8k_dataset' # Folder where you want the files to go

# 3. Create the extraction directory if it doesn't exist
if not os.path.exists(extract_to_path):
    os.makedirs(extract_to_path)

# 4. Extract the zip file
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
    print(f"Successfully extracted to {extract_to_path}")
except FileNotFoundError:
    print(f"File not found at {zip_path}. Please check the path.")
except zipfile.BadZipFile:
    print("The file appears to be a corrupted zip file.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully extracted to /content/drive/MyDrive/flickr8k_dataset


In [16]:
!pip install pycocoevalcap

In [19]:
import transformers
print(f"Transformers Version: {transformers.__version__}")
print(f"LAVIS Model Type: {type(model)}")
print(f"Inner LLM Type: {type(model.llm_model)}")
print(f"Inner LLM Dir: {dir(model.llm_model)[-10:]}") # Check last few attributes

Transformers Version: 4.57.3
LAVIS Model Type: <class 'lavis.models.blip2_models.blip2_vicuna_instruct.Blip2VicunaInstruct'>
Inner LLM Type: <class 'lavis.models.blip2_models.modeling_llama.LlamaForCausalLM'>
Inner LLM Dir: ['tp_size', 'train', 'training', 'type', 'use_kernels', 'vocab_size', 'warn_if_padding_and_no_attention_mask', 'warnings_issued', 'xpu', 'zero_grad']


In [24]:
import os
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

# --- CONFIGURATION ---
DATASET_DIR = '/content/drive/MyDrive/flickr8k_dataset'
IMAGES_DIR = os.path.join(DATASET_DIR, 'Images')
CAPTIONS_FILE = os.path.join(DATASET_DIR, 'captions.txt')
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- GENERATION FUNCTION ---
# (Assuming model/processor are already loaded in memory)
def generate_caption(image):
    prompt = "Describe this image in a short, concise sentence."
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(DEVICE, torch.float16)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            num_beams=5,
            max_new_tokens=30,
            min_length=8,
            repetition_penalty=1.5,
        )
    return processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

# --- EVALUATION LOOP ---
def run_evaluation():
    if not os.path.exists(CAPTIONS_FILE):
        print("❌ Captions file not found.")
        return

    # 1. Load Ground Truths
    print("Loading Ground Truths...")
    df = pd.read_csv(CAPTIONS_FILE)
    df.columns = [c.strip() for c in df.columns]

    # We use ONE format for ALL metrics: Dict of Lists of Strings
    # { 'image_id': ['caption 1', 'caption 2', ...] }
    gts = {}

    for _, row in df.iterrows():
        img_name = row['image']
        caption = str(row['caption'])

        if img_name not in gts: gts[img_name] = []
        gts[img_name].append(caption)

    # 2. Test Subset (Last 50 images)
    test_images = list(gts.keys())[-50:]

    res = {}

    print(f"🚀 Running InstructBLIP on {len(test_images)} images...")

    # 3. Generation Loop
    for img_name in tqdm(test_images):
        img_path = os.path.join(IMAGES_DIR, img_name)
        if not os.path.exists(img_path): continue

        try:
            image = Image.open(img_path).convert("RGB")
            pred = generate_caption(image)

            # Store as List of Strings
            res[img_name] = [pred]

        except Exception as e:
            print(f"Error on {img_name}: {e}")

    # 4. Compute Scores
    if res:
        # Filter Ground Truths to match ONLY the tested images
        # (The scoring library crashes if keys don't match exactly)
        gts_subset = {k: gts[k] for k in res.keys()}

        print("\n📊 --- InstructBLIP Final Scores ---")

        # A. BLEU
        scorer_bleu = Bleu(4)
        score, _ = scorer_bleu.compute_score(gts_subset, res)
        print(f"Bleu-1: {score[0]:.4f}")
        print(f"Bleu-4: {score[3]:.4f}")

        # B. ROUGE
        scorer_rouge = Rouge()
        score, _ = scorer_rouge.compute_score(gts_subset, res)
        print(f"ROUGE_L: {score:.4f}")

        # C. CIDEr (Now uses the same string input!)
        scorer_cider = Cider()
        score, _ = scorer_cider.compute_score(gts_subset, res)
        print(f"CIDEr:   {score:.4f}")

    else:
        print("❌ No results generated.")

# Run
run_evaluation()

Loading Ground Truths...
🚀 Running InstructBLIP on 50 images...


100%|██████████| 50/50 [00:58<00:00,  1.16s/it]


📊 --- InstructBLIP Final Scores ---
{'testlen': 1451, 'reflen': 774, 'guess': [1451, 1401, 1351, 1301], 'correct': [521, 211, 75, 24]}
ratio: 1.8746770025815573
Bleu-1: 0.3591
Bleu-4: 0.0863
ROUGE_L: 0.3099
CIDEr:   0.1022


In [25]:
!wget -q https://github.com/Delphboy/karpathy-splits/raw/main/dataset_flickr8k.json -O dataset_flickr8k.json
print("✅ Karpathy Split JSON downloaded.")

✅ Karpathy Split JSON downloaded.


In [26]:
import torch
import json
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

# --- CONFIGURATION ---
DATASET_JSON = 'dataset_flickr8k.json' # The file we just downloaded
DATASET_DIR = '/content/drive/MyDrive/flickr8k_dataset'
IMAGES_DIR = os.path.join(DATASET_DIR, 'Images')
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- 1. LOAD MODEL (InstructBLIP Vicuna 7B) ---
# This is the SOTA model that fits in Colab Pro
MODEL_ID = "Salesforce/instructblip-vicuna-7b"

print(f"🔄 Loading {MODEL_ID}...")
processor = InstructBlipProcessor.from_pretrained(MODEL_ID)
model = InstructBlipForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
).to(DEVICE)
print("✅ Model Loaded.")

# --- 2. GENERATION FUNCTION (Zero-Shot) ---
def generate_caption(image):
    # SOTA Prompt for InstructBLIP (Zero-Shot)
    # The paper recommends a direct instruction rather than "A photo of..."
    prompt = "Write a short description for the image."

    inputs = processor(images=image, text=prompt, return_tensors="pt").to(DEVICE, torch.float16)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            num_beams=5,
            max_new_tokens=30, # Keep it short to match Flickr style
            min_length=8,
            repetition_penalty=1.5,
            length_penalty=1.0,
        )

    return processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

# --- 3. EVALUATION LOOP ---
def run_zero_shot_eval():
    # A. Parse Karpathy JSON to get the TEST split
    with open(DATASET_JSON, 'r') as f:
        data = json.load(f)

    # Filter for 'test' split images only
    test_images = [img for img in data['images'] if img['split'] == 'test']
    print(f"ℹ️ Found {len(test_images)} images in Karpathy Test Split.")

    # B. Prepare Ground Truths (Dict of list of strings)
    # Format: {'image_filename': ['caption 1', 'caption 2', ...]}
    gts = {}
    for img in test_images:
        filename = img['filename']
        captions = [c['raw'] for c in img['sentences']]
        gts[filename] = captions

    # C. Run Inference
    res = {}
    print(f"🚀 Running Zero-Shot Inference on {len(test_images)} images...")

    for item in tqdm(test_images):
        filename = item['filename']
        img_path = os.path.join(IMAGES_DIR, filename)

        if not os.path.exists(img_path):
            # Fallback: Flickr8k filenames in JSON sometimes differ slightly from local files
            # Try removing or adding folder prefixes if needed.
            # For now, we skip missing files to avoid crashes.
            continue

        try:
            image = Image.open(img_path).convert("RGB")
            pred = generate_caption(image)
            res[filename] = [pred]
        except Exception as e:
            print(f"Error on {filename}: {e}")

    # D. Compute Scores
    if res:
        # Filter Ground Truths to match only successfully generated images
        gts_subset = {k: gts[k] for k in res.keys()}

        print("\n📊 --- InstructBLIP Zero-Shot Results (Karpathy Split) ---")

        # BLEU
        scorer_bleu = Bleu(4)
        score, _ = scorer_bleu.compute_score(gts_subset, res)
        print(f"Bleu-1: {score[0]:.4f}")
        print(f"Bleu-4: {score[3]:.4f}")

        # ROUGE
        scorer_rouge = Rouge()
        score, _ = scorer_rouge.compute_score(gts_subset, res)
        print(f"ROUGE_L: {score:.4f}")

        # CIDEr
        scorer_cider = Cider()
        score, _ = scorer_cider.compute_score(gts_subset, res)
        print(f"CIDEr:   {score:.4f}")
    else:
        print("❌ No results generated.")

run_zero_shot_eval()

🔄 Loading Salesforce/instructblip-vicuna-7b...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model Loaded.
ℹ️ Found 1000 images in Karpathy Test Split.
🚀 Running Zero-Shot Inference on 1000 images...


100%|██████████| 1000/1000 [11:01<00:00,  1.51it/s]



📊 --- InstructBLIP Zero-Shot Results (Karpathy Split) ---
{'testlen': 17001, 'reflen': 15022, 'guess': [17001, 16001, 15001, 14001], 'correct': [8009, 3612, 1640, 705]}
ratio: 1.1317401144986599
Bleu-1: 0.4711
Bleu-4: 0.1555
ROUGE_L: 0.3729
CIDEr:   0.3671
